In [6]:
# importing necesssary libraries

import pandas as pd
from bs4 import BeautifulSoup
import requests

import numpy as np
from pandas.io.json import json_normalize
import requests
import geocoder
from geopy.geocoders import Nominatim

from sklearn.cluster import KMeans

import folium

ModuleNotFoundError: No module named 'geocoder'

In [2]:
# accessing wikipedia
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
postal_url = requests.get(url)

In [3]:
#Converting html data to text and appending to a list
table_contents=[]
html_data = postal_url.text
soup = BeautifulSoup(html_data,'html.parser')
table=soup.find('table')
table
for row in table.findAll('td'):
    cell = {'PostalCode':'PostalCode', 'Borough':'Borough', 'Neighborhood':'Neighborhood'}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [4]:
#Creating pandas dataframe from the previous list
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [5]:
df.shape

(103, 3)

In [ ]:
# Using the given csv file
df_lat_long = pd.read_csv("./Geospatial_Coordinates.csv")
df_lat_long.head()

In [ ]:


# merging the pandas dataframe of the postal codes with the lat/long dataframe
df_postal_toronto = pd.merge(df, df_lat_long, how='left', left_on = 'PostalCode', right_on = 'Postal Code')
df_postal_toronto

# removing the duplicated "Postal Code" column
df_postal_toronto.drop("Postal Code", axis=1, inplace=True)
df_postal_toronto.head()



In [ ]:
# installing geocoder library
!pip install geocoder

In [ ]:
import numpy as np
from pandas.io.json import json_normalize
import requests
import geocoder
from geopy.geocoders import Nominatim

from sklearn.cluster import KMeans

import folium

In [ ]:
# create map of the Toronto neighborhoods using latitude and longitude values

# Getting the coordinates of Toronto
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.arcgis('{}, Toronto, Ontario')
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_postal_toronto['Latitude'], df_postal_toronto['Longitude'], df_postal_toronto['Borough'], df_postal_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
df_postal_toronto.iloc[2]

In [ ]:
address = [df_postal_toronto['Latitude'].iloc[2],df_postal_toronto['Longitude'].iloc[2]]
address

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

In [ ]:
CLIENT_ID = 'O23G5TZTJ0MO4BRKYXJOCRJYR4QBYRSAFTDRW5JOOXXOHS2N' # your Foursquare ID
CLIENT_SECRET = 'B355JQSCI2EGDKOCCIN2QD3NBWLFT0FFOQ4LUBABXQLEPTSG' # your Foursquare Secret
ACCESS_TOKEN = 'TOBURUFG0COEDNQZH4RCISEKUDJFGW1IW2JHSZGTP1S2W3HX' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
search_query = 'Italian'
radius = 1000
print(search_query + ' .... OK!')

In [ ]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

In [ ]:
results = requests.get(url).json()
results

In [ ]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

In [ ]:
dataframe_filtered.name

In [ ]:


venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Neighborhood example: Regent Park, Harbourfront!',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

